<a href="https://colab.research.google.com/github/vidyasager162/LULC-Classification-GEE/blob/main/Modified_LULC_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

In [2]:
cloud_project = 'ee-vidyasager162'

try:
  ee.Initialize(project=cloud_project)
except:
  ee.Authenticate()
  ee.Initialize(project=cloud_project)

In [3]:
import json
import random

In [4]:
# Define the region of interest

roi = ee.Geometry.Polygon(
    [[[11.946657505084461, 43.180502006536486],
      [11.946657505084461, 43.08078081925808],
      [12.071626987506336, 43.08078081925808],
      [12.071626987506336, 43.180502006536486]]])

# Load the dataset and feature collections
dataset = ee.Image("users/vidyasager162/DATASET_COMPOSITION")
newfc = ee.FeatureCollection("users/andreatassi23/Training_point")
valpnts = ee.FeatureCollection("users/andreatassi23/valpnts_AT")
merged = newfc.merge(valpnts)

classes = [0, 1, 2, 3, 4, 5, 6]

In [5]:
# seed = random.randint(0, 1e6)
newfc = ee.FeatureCollection([])
train = ee.FeatureCollection([])
def movePointsForClass(classValue):
  global merged, newfc, seed, valpnts
  classPoints = merged.filter(ee.Filter.eq('LULC', classValue))
  # classPointsWithRandom = classPoints.randomColumn('random', seed)
  classPointsWithRandom = classPoints.randomColumn('random')
  pointsToMove = classPointsWithRandom.sort('random').limit(20)

  pointsToMoveIndices = pointsToMove.aggregate_array('system:index').getInfo()
  exclusionFilter = ee.Filter.Not(
      ee.Filter.inList('system:index', pointsToMoveIndices)
  )
  merged = merged.filter(ee.Filter.Or(
      ee.Filter.neq('LULC', classValue),
      exclusionFilter
  ))

  newfc = newfc.merge(pointsToMove)

for classValue in classes:
  movePointsForClass(classValue)

valpnts = merged

print(dataset.getInfo())
print("Newfc: ", newfc.getInfo())
print("Newfc Size: ", newfc.size().getInfo(), "\n")
print("ValPoints: ", valpnts.getInfo(), "\n")

def movePoints(classValue):
  global newfc, train
  classPoints = newfc.filter(ee.Filter.eq('LULC', classValue))
  classPointsWithRandom = classPoints.randomColumn('random')
  pointsToMove = classPointsWithRandom.sort('random').limit(20)

  train = train.merge(pointsToMove)

for classValue in classes:
  movePoints(classValue)

# train = newfc

{'type': 'Image', 'bands': [{'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1392, 1111], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 11.946605131977105, 0, -8.983152841195215e-05, 43.180578403170806]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1392, 1111], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 11.946605131977105, 0, -8.983152841195215e-05, 43.180578403170806]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1392, 1111], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 11.946605131977105, 0, -8.983152841195215e-05, 43.180578403170806]}, {'id': 'B6', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [1392, 1111], 'crs': 'EPSG:4326', 'crs_transform': [8.983152841195215e-05, 0, 11.946605131977105, 0, -8.983152841195215e-05, 43.180578403170806]}, {'id': 'B8', 'data_type': {'type': 'PixelTyp

In [6]:
print("Training: ", train.getInfo(), "\n")
print("Training Size: ", train.size().getInfo(), "\n")
print("Validation Size: ", valpnts.size().getInfo(), "\n")



# Buffer function for training data
buffer = lambda feature: feature.buffer(10)
train = train.map(buffer)

print(train.getInfo(), "\n")

Training:  {'type': 'FeatureCollection', 'columns': {}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [12.023055379846896, 43.16817097967903]}, 'id': '1_1_1_1_1_1_2_1_1_1_1_1_1_2_2_00000000000000000077', 'properties': {'LULC': 0, 'random': 0.0977675237387372}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [12.033355062464082, 43.17768551593718]}, 'id': '1_1_1_1_1_1_2_1_1_1_1_1_1_2_2_00000000000000000069', 'properties': {'LULC': 0, 'random': 0.16773447644157535}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [12.027861898401584, 43.17618331928497]}, 'id': '1_1_1_1_1_1_2_1_1_1_1_1_1_2_2_0000000000000000006a', 'properties': {'LULC': 0, 'random': 0.19334571044128734}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [12.02442867086252, 43.164540042231046]}, 'id': '1_1_1_1_1_1_2_1_1_1_1_1_1_2_2_00000000000000000078', 'properties': {'LULC': 0, 'random': 0.19939822085014602}}, {'type': 'Feature', 'geometr

In [7]:
# Define classifier parameters
# numTrees = 50

classifiers = {
    'RF': {
        'num_trees_options': [10, 20, 50, 100, 150, 200, 500],
        'max_depth_options': [10, 20, 30, 40, 50, 60, None]
    },
    'SVM': {
        'gamma_options': [0.1, 0.2, 0.25, 0.5, 1],
        'cost_options': [1, 20, 30, 40, 50]
    }
}
# num_trees_options = [10, 20, 50, 100, 150, 200, 500]
# max_depth_options = [10, 20, 30, 40, 50, 60, None]

# gamma_options = [0.1, 0.2, 0.25, 0.5, 1]
# cost_options = [1, 20, 30, 40, 50]

best_accuracy_rf_pb = 0
best_accuracy_svm_pb = 0
best_accuracy_rf_oo = 0
best_accuracy_svm_oo = 0

best_params_rf_pb = {}
best_params_svm_pb = {}
best_params_rf_oo = {}
best_params_svm_oo = {}

In [8]:
# Helper Functions

def compute_metrics(testAccuracy):
    precision = testAccuracy.producersAccuracy()
    recall = testAccuracy.consumersAccuracy().toList().get(0)
    recall = ee.Array(recall)
    f1 = testAccuracy.fscore(1)

    avg_precision = precision.reduce(ee.Reducer.mean(), [0]).toList().get(0)
    avg_precision = ee.Array(avg_precision).get([0])
    avg_recall = recall.reduce(ee.Reducer.mean(), [0]).toList().get(0)
    avg_f1 = f1.reduce(ee.Reducer.mean(), [0]).toList().get(0)

    metrics = ee.Dictionary({
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'avg_precision': avg_precision,
        'avg_recall': avg_recall,
        'avg_f1': avg_f1,
        'overall_accuracy': testAccuracy.accuracy()
    })

    return metrics

def print_metrics(testAccuracy, approach):
    metrics = compute_metrics(testAccuracy)
    print(f'{approach} APPROACH: Confusion Matrix ', testAccuracy.getInfo())
    print(json.dumps(metrics.getInfo()), "\n")

In [9]:
# Pixel-based approach

# Mandatory normalization of the input bands for SVM
image = dataset
band_names = image.bandNames()
minMax = image.reduceRegion(reducer=ee.Reducer.minMax(), geometry=roi, scale=10, maxPixels=1e10)

def unitScale(band):
  band = ee.String(band)
  min_val = ee.Number(minMax.get(band.cat('_min')))
  max_val = ee.Number(minMax.get(band.cat('_max')))
  return image.select(band).unitScale(min_val, max_val)

dataset = ee.ImageCollection(band_names.map(unitScale)).toBands().rename(band_names)
bands = dataset.bandNames()

training = dataset.select(bands).sampleRegions(
    collection=train,
    properties=['LULC'],
    scale=10
)

testing = dataset.select(bands).sampleRegions(
  collection=valpnts,
  properties=['LULC'],
  scale=10
)

for classifer, parameter_dict in classifiers.items():
  if classifer == 'RF':
    num_trees_options = parameter_dict['num_trees_options']
    max_depth_options = parameter_dict['max_depth_options']
    for num_trees in num_trees_options:
      for max_depth in max_depth_options:
        rf_classifier = ee.Classifier.smileRandomForest(
            numberOfTrees = num_trees,
            maxNodes = max_depth
        ).train(
          features=training,
          classProperty='LULC',
          inputProperties=bands
        )

        # Classify and validate using RF
        classified_rf = dataset.select(bands).classify(rf_classifier).clip(roi).focal_mode()
        classified_test_rf = testing.classify(rf_classifier)
        testAccuracy_rf = classified_test_rf.errorMatrix('LULC', 'classification')
        accuracy_rf = testAccuracy_rf.accuracy().getInfo()

        # Print metrics for RF
        print(f'PIXEL BASED APPROACH: RF Classification: num_trees: {num_trees} and max_depth: {max_depth}')
        print_metrics(testAccuracy_rf, 'PIXEL')

        # Store the best parameters
        if accuracy_rf > best_accuracy_rf_pb:
          best_accuracy_rf_pb = accuracy_rf
          best_params_rf_pb = {
              'num_trees': num_trees,
              'max_depth': max_depth
          }
    print('Best RF Parameters PB:', best_params_rf_pb)
    print('Best RF Accuracy PB:', best_accuracy_rf_pb, "\n")

  elif classifer == 'SVM':
    gamma_options = parameter_dict['gamma_options']
    cost_options = parameter_dict['cost_options']
    for gamma in gamma_options:
      for cost in cost_options:
        svm_classifier = ee.Classifier.libsvm(
            kernelType='RBF',
            gamma=gamma,
            cost=cost
            ).train(training,
                    'LULC',
                    bands
            )

        # Classify and validate using SVM
        classified_svm = dataset.select(bands).classify(svm_classifier).clip(roi).focal_mode()
        classified_test_svm = testing.classify(svm_classifier)
        testAccuracy_svm = classified_test_svm.errorMatrix('LULC', 'classification')
        accuracy_svm = testAccuracy_svm.accuracy().getInfo()

        # Print metrics for SVM
        print(f'PIXEL BASED APPROACH: SVM Classification: gamma: {gamma} and cost: {cost}')
        print_metrics(testAccuracy_svm, 'PIXEL')

        # Store the best parameters
        if accuracy_svm > best_accuracy_svm_pb:
          best_accuracy_svm_pb = accuracy_svm
          best_params_svm_pb = {
              'gamma': gamma,
              'cost': cost
          }
    print('Best SVM Parameters PB:', best_params_svm_pb)
    print('Best SVM Accuracy PB:', best_accuracy_svm_pb, "\n")

PIXEL BASED APPROACH: RF Classification: num_trees: 10 and max_depth: 10
PIXEL APPROACH: Confusion Matrix  [[69, 1, 0, 0, 0, 0, 0], [0, 19, 3, 1, 1, 1, 0], [0, 2, 20, 1, 5, 2, 0], [0, 7, 5, 115, 3, 14, 0], [0, 0, 0, 0, 6, 1, 0], [0, 0, 7, 7, 4, 31, 0], [0, 0, 0, 0, 1, 0, 54]]
{"avg_f1": 0.7482894604928875, "avg_precision": 0.8118008805253704, "avg_recall": 0.726667628754982, "f1": [0.9928057553956835, 0.7037037037037037, 0.6153846153846153, 0.8582089552238806, 0.4444444444444444, 0.6326530612244898, 0.9908256880733944], "overall_accuracy": 0.8263157894736842, "precision": [[0.9857142857142858], [0.76], [0.6666666666666666], [0.7986111111111112], [0.8571428571428571], [0.6326530612244898], [0.9818181818181818]], "recall": [1, 0.6551724137931034, 0.5714285714285714, 0.9274193548387096, 0.3, 0.6326530612244898, 1]} 

PIXEL BASED APPROACH: RF Classification: num_trees: 10 and max_depth: 20
PIXEL APPROACH: Confusion Matrix  [[69, 1, 0, 0, 0, 0, 0], [0, 21, 3, 0, 1, 0, 0], [0, 1, 23, 1, 4, 1

In [ ]:
# Define algorithm Parameters

compactness_options = [0, 0.1, 0.5, 1, 2]
connectivity_options = [4, 8]
neighborhoodSize_options = [32, 64, 128, 256, 512]
size_segmentation_options = [4, 5, 8, 10, 15, 16, 20, 32]
glcm_size_options = [1, 2, 3, 5, 7]

# # Define the superpixel seed location spacing, in pixels
# size_segmentation = 15

# Define the GLCM indices used in input for the PCA
glcm_bands = [
    'gray_asm',
    'gray_contrast',
    'gray_corr',
    'gray_ent',
    'gray_var',
    'gray_idm',
    'gray_savg'
]

def scale_bands(name):
  name = ee.String(name)
  band = image.select(name)
  min_val = ee.Number(minMax.get(name.cat('_min')))
  max_val = ee.Number(minMax.get(name.cat('_max')))
  return band.unitScale(min_val, max_val)

# Function to return the Principal Components in the region as a new image
def getPrincipalComponents(centered, scale, region):
  arrays = centered.toArray()
  covar = arrays.reduceRegion(reducer=ee.Reducer.centeredCovariance(), geometry=region, scale=scale)
  covarArray = ee.Array(covar.get('array'))
  eigens = covarArray.eigen()
  eigenValues = eigens.slice(1, 0, 1)
  eigenVectors = eigens.slice(1, 1)

  arrayImage = arrays.toArray(1)
  principalComponents = ee.Image(eigenVectors).matrixMultiply(arrayImage)

  sdImage = ee.Image(eigenValues.sqrt()).arrayProject([0]).arrayFlatten([getNewBandNames('sd')])

  return principalComponents.arrayProject([0]).arrayFlatten([getNewBandNames('pc')]).divide(sdImage)

def scale_band(name):
  band_name = ee.String(name)
  min_val = ee.Number(minMax.get(band_name.cat('_min')))
  max_val = ee.Number(minMax.get(band_name.cat('_max')))
  band = final_bands.select(band_name)
  return band.unitScale(min_val, max_val).rename(band_name)

# Create and rescale a grayscale image for GLCM
gray = dataset.expression(
    '(0.3 * NIR) + (0.59 * R) + (0.11 * G)', {
        'NIR': dataset.select('B8'),
        'R': dataset.select('B4'),
        'G': dataset.select('B3')
    }).rename('gray')

In [ ]:
# Object-based approach

for size_segmentation in size_segmentation_options:
  for connectivity in connectivity_options:
    for compactness in compactness_options:
      for neighborhoodSize in neighborhoodSize_options:
        for glcm_size in glcm_size_options:
          # Segmentation using a SNIC approach
          seeds = ee.Algorithms.Image.Segmentation.seedGrid(size_segmentation)
          snic = ee.Algorithms.Image.Segmentation.SNIC(image=dataset, compactness=0, connectivity=8, neighborhoodSize=256, seeds=seeds)
          glcm = gray.unitScale(0, 0.30).multiply(100).toInt().glcmTexture(size=2)

          # Scaling the GLCM bands before PCA
          image = glcm.select(glcm_bands)
          minMax = image.reduceRegion(
              reducer=ee.Reducer.minMax(),
              geometry=roi,
              scale=10,
              maxPixels=1e10
          )

          glcm = ee.ImageCollection.fromImages(
              image.bandNames().map(scale_bands)
          ).toBands().rename(image.bandNames())

          # Applying PCA
          scale = glcm.projection().nominalScale()
          bandNames = glcm.bandNames()

          meanDict = glcm.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale= scale, maxPixels=1e16)

          means = ee.Image.constant(meanDict.values(bandNames))
          centered = glcm.subtract(means)

          getNewBandNames = lambda prefix: ee.List.sequence(1, bandNames.length()).map(lambda b: ee.String(prefix).cat(ee.Number(b).int()))

          pcImage = getPrincipalComponents(centered, scale, roi)

          clusters_snic = snic.select('clusters')
          clusters_snic = clusters_snic.reproject(crs=clusters_snic.projection(), scale=10)

          new_feature = clusters_snic.addBands(pcImage.select("pc1"))

          new_feature_mean = new_feature.reduceConnectedComponents(reducer=ee.Reducer.mean(), labelBand='clusters')

          final_bands = new_feature_mean.addBands(snic)
          predictionBands = final_bands.bandNames().remove('clusters')

          # Mandatory normalization of the input bands for SVM
          minMax = final_bands.reduceRegion(reducer=ee.Reducer.minMax(), geometry=roi, scale=10, maxPixels=1e10)

          # Apply scaling function to each band
          scaled_bands = ee.ImageCollection.fromImages(
              final_bands.bandNames().map(scale_band)
          ).toBands().rename(final_bands.bandNames())

          # Replace final_bands with scaled_bands
          final_bands = scaled_bands

          training_geobia = final_bands.select(predictionBands).sampleRegions(
              collection=train,
              properties=['LULC'],
              scale=10
          )

          classifier_geobia = final_bands.select(predictionBands).sampleRegions(
            collection=valpnts,
            properties=['LULC'],
            scale=10
          )

          for classifer, parameter_dict in classifiers.items():
            if classifer == 'RF':
              num_trees_options = parameter_dict['num_trees_options']
              max_depth_options = parameter_dict['max_depth_options']
              for num_trees in num_trees_options:
                for max_depth in max_depth_options:
                  rf_classifier = ee.Classifier.smileRandomForest(
                      numberOfTrees = num_trees,
                      maxNodes = max_depth
                  ).train(
                    features=training_geobia,
                    classProperty='LULC',
                    inputProperties=predictionBands
                  )

                  # Classify and validate using RF
                  classy_RF = final_bands.select(predictionBands).classify(rf_classifier)
                  classy_RF = classy_RF.reproject(crs=classy_RF.projection(), scale=10)
                  classificazione = classifier_geobia.classify(rf_classifier)
                  testAccuracy_rf = classificazione.errorMatrix('LULC', 'classification')
                  accuracy_rf = testAccuracy_rf.accuracy().getInfo()

                  # Store the best parameters
                  if accuracy_rf > best_accuracy_rf_oo:
                    best_accuracy_rf_oo = accuracy_rf
                    best_params_rf_oo = {
                        'num_trees': num_trees,
                        'max_depth': max_depth
                    }

print('Best RF Parameters OO:', best_params_rf_oo)
print('Best RF Accuracy OO:', best_accuracy_rf_oo)

for gamma in gamma_options:
  for cost in cost_options:
    svm_classifier = ee.Classifier.libsvm(
        kernelType='RBF',
        gamma=gamma,
        cost=cost
        ).train(training_geobia, 'LULC', predictionBands)

    # Classify and validate using SVM
    classy_SVM = final_bands.select(predictionBands).classify(svm_classifier)
    classy_SVM = classy_SVM.reproject(crs=classy_SVM.projection(), scale=10)
    classificazione = classifier_geobia.classify(svm_classifier)
    testAccuracy_svm = classificazione.errorMatrix('LULC', 'classification')
    accuracy_svm = testAccuracy_svm.accuracy().getInfo()

    # Store the best parameters
    if accuracy_svm > best_accuracy_svm_oo:
      best_accuracy_svm_oo = accuracy_svm
      best_params_svm_oo = {
          'gamma': gamma,
          'cost': cost
      }
print('Best SVM Parameters OO:', best_params_svm_oo)
print('Best SVM Accuracy OO:', best_accuracy_svm_oo)

# Print metrics for Object-based RF
# print('OBJECT BASED APPROACH: RF Classification')
# print_metrics(testAccuracy_rf, 'OBJECT')

# Print metrics for Object-based SVM
# print('OBJECT BASED APPROACH: SVM Classification')
# print_metrics(testAccuracy_svm, 'OBJECT')